## [라이브러리 설치] 3D model 처리에 특화된 라이브러리
- trimesh : triangular meshe model 사용 및 적재에 특화된 라이브러리입니다. Model10 dataset은 CAD로 구현된 object인 Mesh model로 제공됩니다. Mesh model에 특화된 라이브러리 trimesh를 이용하여 Mesh model에서 point cloud로 변환하여 제공합니다.
- Point Cloud Library(pcl) : 2D/3D image와 point cloud 처리를 위해 제작된 라이브러리입니다. point cloud를 이용한 feature extraction, segmentation, noise filtering 등 다양한 방법론에 대한 알고리즘을 제공하며, 해당 텀프로젝트에서는 point cloud의 normal vector를 추론하기위해 사용합니다.
- plotly : 데이터 분석 및 시각화 라이브러리 입니다. ModelNet dataset의 Mesh model과 point cloud를 Colab-notebook에서 active 3d plot으로 visualize 하기 위해 사용합니다.

In [1]:
!pip install --upgrade pip
!apt update 

# kaggle
!pip install kaggle

# trimesh
!pip install trimesh

# pcl
!sudo add-apt-repository ppa:sweptlaser/python3-pcl -y #Python3 Only??
!sudo apt update
!sudo apt install python3-pcl

# plotly
!pip install plotly

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/sweptlaser/python3-pcl/ubuntu bionic InRelease
Get:15 http

In [2]:
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c 2020mltermproject3dclassification && unzip /content/data.json.zip

Saving kaggle.json to kaggle.json
sample_submit.csv: Skipping, found more recently modified local copy (use --force to force download)
data.json.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/data.json.zip
replace data.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [4]:
import os
import glob
from tqdm import tqdm

import tensorflow as tf

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objs as go
from matplotlib import pyplot as plt

import trimesh
import pcl


# ModelNet10

>ModelNet10의 클래스는 10가지 종류(옷장, 책상, 욕조, 나이트 스탠드, 모니터, 테이블, 의자, 소파, 침대, 변기)로 구성됩니다.
>
>원본 데이터는 아래와 같은 mesh model로 제공되며 아래의 cell에서 시각화된 Mesh model을 확인 할 수 있습니다.
> 
> 텀 프로젝트에서는 trimesh로 가공한 1024개의 point cloud로 변환하여 제공합니다.

In [5]:
# ModelNet10 download
DATA_DIR = tf.keras.utils.get_file(
    "modelnet.zip",
    "http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip",
    extract=True,
)
DATA_DIR = os.path.join(os.path.dirname(DATA_DIR), "ModelNet10")

In [6]:
# chair_0004의 mesh model 시각화
mesh = trimesh.load(os.path.join(DATA_DIR, "chair/train/chair_0004.off"))

mesh.show()

## 데이터 불러오기
> trimesh로 가공된 1024개의 point cloud는 아래의 data tree로 구성된 json 파일로 제공되어 집니다.
>
> 하단의 cell을 통해 point cloud로 구성된 model의 시각화를 확인 할 수 있습니다.

Data tree
```
+--data
|  +--train
|     +--points : 3991개의 (1024,3) point cloud
|     +--label : 0~9
|  +--test
|     +--points : 908개의 (1024,3) point cloud
|  +--class map : (0:옷장, 1:책상, 2:욕조, 3:나이트 스탠드, 4:모니터, 5:테이블, 6:의자, 7:소파, 8:침대, 9:변기)
```


In [3]:
import json

# 데이터 로드
with open('/content/data.json') as json_file:
    data = json.load(json_file)

In [7]:
# point cloud는 1024개의 (x,y,z)를 가집니다.
data['train'][0]['points'][:10]

[[-3.1336569241001673, -0.04298979133216996, -3.097491411468202],
 [1.409545912464028, 1.9627392518070748, 1.598013630980347],
 [-1.5264733313105128, 12.32002, 1.0291060132320737],
 [0.6350462238345201, -0.24999927562503776, 3.0767061136363076],
 [-3.1012942963771137, 6.0839247881924905, -7.021356215778342],
 [2.492453751082173, 0.3193954794480329, -7.072872161099004],
 [-2.39195482440834, -8.581290175872093, 0.5270379730297408],
 [-2.9352544990122653, -5.547947507394143, -2.1279512694905165],
 [-1.516750791376341, -12.32002, 2.927179904532773],
 [-2.167146590948964, 9.230309857086112, 1.6256277005125503]]

In [8]:
# data.json 출력 예시
print(f'class map : {data["class map"]}')
print(f'train 0 번째 point cloud : {data["train"][0]["points"][:10]}')
print(f'train 0 번째 label : {data["train"][0]["label"]}')
label = data["train"][0]["label"]

class map : {'0': 'monitor', '1': 'bathtub', '2': 'table', '3': 'bed', '4': 'chair', '5': 'toilet', '6': 'desk', '7': 'sofa', '8': 'night_stand', '9': 'dresser'}
train 0 번째 point cloud : [[-3.1336569241001673, -0.04298979133216996, -3.097491411468202], [1.409545912464028, 1.9627392518070748, 1.598013630980347], [-1.5264733313105128, 12.32002, 1.0291060132320737], [0.6350462238345201, -0.24999927562503776, 3.0767061136363076], [-3.1012942963771137, 6.0839247881924905, -7.021356215778342], [2.492453751082173, 0.3193954794480329, -7.072872161099004], [-2.39195482440834, -8.581290175872093, 0.5270379730297408], [-2.9352544990122653, -5.547947507394143, -2.1279512694905165], [-1.516750791376341, -12.32002, 2.927179904532773], [-2.167146590948964, 9.230309857086112, 1.6256277005125503]]
train 0 번째 label : 0


In [9]:
# chair_0004의 point cloud 시각화
import plotly.express as px
import plotly.graph_objs as go

# chair/train/chair_0004.off
points = np.array(data['train'][1478]['points'])

fig = go.Figure(data=[go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2],
                                   mode='markers',  marker=dict( size=3) ) ])
fig.show()

# DataLoader

In [10]:
from torch.utils.data import Dataset

class Model10Dataset(Dataset):
    def __init__(self, data, mode='train'):
        self.data = data[mode]
        self.class_map = data['class map']
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        xyz = np.array(self.data[idx]['points'], dtype=np.float32)
        return {'points':xyz, 'label':self.data[idx]['label']}


> 이번 텀 프로젝트에서는 3D 데이터를 point의 변화에 불변성을 가진 1D vector를 만들어 각 클래스간 분류를 해볼 예정입니다.



# [Empty Module #1] Normal Histogram

##### 목표 : point_cloud를 정규화하고 normal vector를 추론한다.
- [1] `numpy.linalg.norm`를 이용하여 x, y, z에 대한 L2 norm을 구하고 L2 norm을 이용한 정규화를 한다. 
- [2] pcl을 이용한 normal vector estimation을 위해 point cloud를  `pcl.PointCloud()`로 변환한다. `pcl.PointCloud()`는 `np.float32`인 `np.array`를 인자로 받는다.


> ### Normal vector 추론 파이프라인
> 1. point cloud p의 Nearst Neighbor p_k를 획득합니다.
> 2. p와 샘플링된 p_k간의 covariance matrix의 C를 만듭니다.
> 3. C로부터 eigenvalues와 eigenvectors를 추정합니다.
> 4. eigenvalues를 기준으로 eigenvectors 내림차순으로 정렬합니다.
> 5. 가장 작은 값을 가진 eigenvalues의 eigenvectors 널리 퍼진 값이기에 normals vector로 표현됩니다.

`pcl::Point_Cloud::make_NormalEstimation()`와 `pcl::Point_Cloud::make_NormalEstimation::set_KSearch()`를 이용하여 point cloud로부터  normal vector 추론이 가능하도록 pcl 라이브러리에 구현되어 있다.

```
    input 
    point_array : np.array, (1024, 3) 차원을 가진 point cloud
    
    output
    normals : pcl.PointCloud_Normals, point cloud로부터 추정된 normal vector 
```


In [11]:
# -------------------------------------
# [Empty Module #1] Normal Vector Estimation
# -------------------------------------
def normal_estimation(point_array, k):
    # ------------------------------------------------------------
    # 구현 가이드라인 
    # ------------------------------------------------------------
    # [1] point cloud x,y,z에 대한 정규화
    #     point_array의 차원 : (1024, 3)
    #     정규화에 사용되는 함수 : numpy.linalg.norm, (L2 norm을 사용)
    # [2] raw data -> pcl 변환. e.g. point = pcl.PointCloud(np.zeros((1024, 3)))
    # ------------------------------------------------------------
    # [1]

    point_array_std = np.linalg.norm(point_array,ord=2)
    point_array = point_array/point_array_std

    # [2]
    point_cloud = pcl.PointCloud(point_array)

    # Normals vector 추정 알고리즘
    feature = point_cloud.make_NormalEstimation()

    # Nearst Neighbor (Octree Search)
    feature.set_KSearch(k)

    # Compute
    normals = feature.compute()
    return normals.to_array().T

# [Empty Module #2] Compute Normal Histogram
#### 목표 : Point cloud로부터 추론된 normal vector {x, y, z}의 histogram을 만든다.
- [1] input인 nbins, nrage를 인자로 사용한 `np.histogram`을 이용하여 normal vector {x, y, z}의 histgoram을 만든다. 
- [2] `np.concatenate`를 이용하여 {x, y, z}의 histogram을 1d vector로 만든다.
- [3] `numpy.linalg.norm`를 이용하여 [2]에서 만들어진 1d vector에 대한 L1 norm을 구하고 L1 norm을 이용한 정규화를 한다. 
```
    input 
    normal_vector : pcl.PointCloud_Normals, point cloud로부터 추정된 normal vector. 차원 : (4, 1024)
      => [[x_0, y_0, z_0, 곡률 c_0]....[x_1023, y_1023, z_1023]].T
    nbins : histogram의 bin의 수
    nrange : histogram의 범위

    output
    normed_features : np.array, 정규화된 normal vector의 x, y, z 히스토그램
```

In [12]:
# -------------------------------------
# [Empty Module #2] Compute Normal Histogram
# -------------------------------------
def compute_normal_histograms(normal_vector, nbins=32, nrange=(-1,1)):

    # ------------------------------------------------------------
    # 구현 가이드라인 
    # ------------------------------------------------------------
    # [1] normal_vector_{x, y, z} 축에 대한 histogram을 쌓습니다.
    #     normal_vector 차원 : (4, 1024) -> (num of point_cloud, (x, y, z, c)) 
    #     사용되는 함수 : np.histogram(), <- nbins, nrange
    # [2] normal_vector_{x, y, z} 축에 대한 histogram을 concat 합니다.
    #     사용되는 함수 : np.concatenate()
    # [3] 각 값들의 histogram을 concat한 hist_concat 정규화
    #     정규화에 사용되는 함수 : numpy.linalg.norm, (L1 norm을 사용)
    #     output : normed_features
    # ------------------------------------------------------------

    # [1]


    normal_vector = normal_vector.T

    norm_x_vals = normal_vector[:,0]
    norm_y_vals = normal_vector[:,1]
    norm_z_vals = normal_vector[:,2]

    norm_x_hist = np.histogram(norm_x_vals, bins=nbins, range=nrange)
    norm_y_hist = np.histogram(norm_y_vals, bins=nbins, range=nrange)
    norm_z_hist = np.histogram(norm_z_vals, bins=nbins, range=nrange) 
    # [2]
    #print(z_bins)
    #print(hist_concat)
    hist_concat = np.concatenate([norm_x_hist[0], norm_y_hist[0], norm_z_hist[0]]).astype(np.float32)

    # [3]
   
    a = np.linalg.norm(hist_concat,ord=1)
    normed_features = hist_concat / a

    return normed_features

# [Empty Module #3] Dataloader로부터 Normal Histogram 추출
#### 목표 : 데이터 셋으로부터 3D model을 Normal Histogram을 이용하여 feature vector로 변환한다.
- [1] 3D 모델의 모든 point cloud로부터 Normal Histogram을 추론
     1. point cloud를 np.array(dtype=np.float32)로 변경
     2. point cloud를 `normal_estimation()`를 이용하여 normal vector를 추론
     3. 추론된 normal vector를 `compute_normal_histograms()` 사용하여
        normal vector 각 원소들에 대한 histogram을 추론
     4. 추론된 histogram을 normal_hitogram에 append

```
  input
  dataset : torch.utils.data.Dataset

  output
  normal_hitogram : np.array
  labels : np.array

```

In [13]:
# -------------------------------------
# [Empty Module #3] Dataloader로부터 Normal Histogram 추출
# -------------------------------------
def Normal_Histogram(dataset):

  # ------------------------------------------------------------
  # 구현 가이드라인 
  # ------------------------------------------------------------
  # [1] 3D 모델의 모든 point cloud로부터 Normal Histogram을 추론
  #     1. point cloud를 np.array(dtype=np.float32)로 변경
  #     2. point cloud를 normal_estimation()를 이용하여 normal vector를 추론
  #     3. 추론된 normal vector를 compute_normal_histograms 사용하여
  #        normal vector 각 원소들에 대한 histogram을 추론
  #     4. 추론된 histogram을 normal_hitogram에 append
  # ------------------------------------------------------------

  normal_hitogram = []
  labels = []

  for i, pc in enumerate(tqdm(dataset)):

      # [1]
      pc_1 = np.array(pc['points'],dtype=np.float32) #1
      normal_vector = normal_estimation(pc_1,3)#2
      normal_hist = compute_normal_histograms(normal_vector) #3
      normal_hitogram.append(normal_hist)#4
      labels.append(pc['label'])

  normal_hitogram=np.array(normal_hitogram, dtype=np.float32)
  labels=np.array(labels, dtype=np.float32)
  return normal_hitogram, labels

# [Empty Module #4] SVC를 이용한 분류
- [1] `sklearn.svm.SVC`를 이용한 분류
    1. `Normal_Histogram()`으로부터 추론된 point cloud의 feature representation(1d vector)을 `sklearn.svm.SVC`를 이용하여 classification 을 진행한다.
    2. fit()으로 x_train, y_train에 대한 머신러닝 학습한다.
    3. predict()으로 x_test에 대한 정답을 추론 하여 y_pred 반환한다.

- tip :  [sklearn.pipeline.make_pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html), [sklearn.preprocessing.StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html?highlight=standardscaler#sklearn.preprocessing.StandardScaler)

In [17]:
# -------------------------------------
# [Empty Module #4] SVC를 이용한 분류
# -------------------------------------

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report
from tqdm.notebook import tqdm

# Datalodaer
model10_dataset_tr = Model10Dataset(data, mode='train')
model10_dataset_ts = Model10Dataset(data, mode='test')

# Normal Histgoram
x_train, y_train = Normal_Histogram(model10_dataset_tr)
x_test, _ = Normal_Histogram(model10_dataset_ts)
x_train = x_train.reshape(3991,-1)

std = StandardScaler()
x_train = std.fit_transform(x_train)
x_test = std.fit_transform(x_test)
# ------------------------------------------------------------
# 구현 가이드라인 
# ------------------------------------------------------------
#  [1] 3D model 클래스간 표면의 normal vector 유사성을 이용한
#      Normal Histogram을 SVC를 이용하여 분류
#      y_pred <- SVC를 이용한 test set 예측값  
# ------------------------------------------------------------

# [1]
param_grid={'kernel':('linear', 'rbf'), 'C':[0.1 ,1, 10, 100, 1000]}

clf=GridSearchCV(SVC(class_weight='balanced'), param_grid, cv=5)

x_train = x_train.reshape(-1,1)
clf.fit(x_train,y_train)

y_pred = clf.predict(x_test)

"\nparam_grid={'kernel':('linear', 'rbf'), 'C':[0.1 ,1, 10, 100, 1000]}\n\nclf=GridSearchCV(SVC(class_weight='balanced'), param_grid, cv=5)\n\nx_train = x_train.reshape(-1,1)\nclf.fit(x_train,y_train)\n\ny_pred = clf.predict(x_test)\n"

In [19]:
submit = pd.read_csv('/content/sample_submit.csv')

submit['Category'] = y_pred.astype(np.int)
submit.to_csv( './baseline.csv', index=False)

!kaggle competitions submit -c 2020mltermproject3dclassification -f baseline.csv -m "FPFH"

print(submit)

100% 5.22k/5.22k [00:00<00:00, 20.2kB/s]
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/kaggle/cli.py", line 64, in main
    print(out, end='')
UnicodeEncodeError: 'latin-1' codec can't encode characters in position 31-34: ordinal not in range(256)
      Id  Category
0      0         0
1      1         0
2      2         0
3      3         0
4      4         0
..   ...       ...
903  903         9
904  904         9
905  905         0
906  906         9
907  907         9

[908 rows x 2 columns]
